In [1]:
import os
from typing import List

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction, SentenceTransformerEmbeddingFunction
from dotenv import load_dotenv
from openai import OpenAI
import torch

load_dotenv()

True

## Test the influence of new chunking method on response generation
We hope that, by having smaller chunks, the response generation will be better because less superfluous information is provided

In [2]:
# First get the Embedding Function and ChromaDB collection
# DOCUMENTS MUST HAVE BEEN ADDED TO THE CHROMA IN PREVIOUS NOTEBOOK

#model_name = "camembert-base"
#model_name = "dangvantuan/sentence-camembert-base"
model_name = "text-embedding-ada-002"

match model_name:
    case "text-embedding-ada-002":
        OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
        emb_func = OpenAIEmbeddingFunction(
                        api_key=OPENAI_API_KEY,
                        model_name="text-embedding-ada-002"
                    )

    case other:
        emb_func = SentenceTransformerEmbeddingFunction(
            model_name=model_name,
            device="cuda" if torch.cuda.is_available() else "cpu")


chroma_client = chromadb.PersistentClient(path=os.path.join("./ChromaDB", model_name))

### Test retrieval manually

In [3]:
# Choose a collection name
collection_name = "chunked-pages-v1"
#collection_name = "ladrome-pages-json-v1-Markdown"
# Choose a query
query = "Comment fonctionne la médiathèque ?"
# choose the number of results
n_results = 10
# choose whether or not results are printed
print_results = True

collection = chroma_client.get_or_create_collection(
    collection_name,
    metadata={"hnsw:space": "cosine"},
    embedding_function=emb_func)

result = collection.query(
    query_texts=query,
    n_results=n_results,
)

if print_results:
    for rd, id in zip(result["documents"][0], result["ids"][0]):
        print(f"---------{id}---------")
        print(rd)

Add of existing embedding ID: 10511_0
Add of existing embedding ID: 10516_0
Add of existing embedding ID: 10516_1
Add of existing embedding ID: 10516_2
Add of existing embedding ID: 10516_3
Add of existing embedding ID: 10516_4
Add of existing embedding ID: 10520_0
Add of existing embedding ID: 10520_1
Add of existing embedding ID: 10520_2
Add of existing embedding ID: 10520_3
Add of existing embedding ID: 10520_4
Add of existing embedding ID: 10520_5
Add of existing embedding ID: 10522_0
Add of existing embedding ID: 10524_0
Add of existing embedding ID: 10526_0
Add of existing embedding ID: 10533_0
Add of existing embedding ID: 10537_0
Add of existing embedding ID: 10542_0
Add of existing embedding ID: 10542_1
Add of existing embedding ID: 10565_0
Add of existing embedding ID: 10565_1
Add of existing embedding ID: 10565_2
Add of existing embedding ID: 10565_3
Add of existing embedding ID: 10565_4
Add of existing embedding ID: 10692_0
Add of existing embedding ID: 10694_0
Add of exist

---------10956_0---------
Les médiathèques départementales
Développer la lecture publique sur tout le territoire drômois est un engagement fort du Département. La Médiathèque départementale assure une double mission : 
- accueillir le public dans 5 médiathèques départementales à Valence, Crest, Die, Nyons et Saint Vallier 
- accompagner et soutenir 115 bibliothèques communales (formation, aide technique, prêt de documents, conseil aux élus…)

---------10771_1---------
Vivre sa retraite dans la Drôme
Les médiathèques et bibliothèques
La lecture est un excellent exercice pour le cerveau. Quatre médiathèques départementales sont à votre disposition à Saint-Vallier, Crest, Die et Nyons. En plus des traditionnels livres et revues, elles proposent des documents qui, par leur forme ou leur contenu, peuvent être intéressants à consulter: 
- livres en gros caractères pour le confort de lecture; 
- livres audio 
- documents sur la vie quotidienne: santé, alimentation, droits et démarches… 
- CD 

### Test response generation manually

In [4]:
# Choose a collection name
collection_name = "chunked-pages-v1"
#collection_name = "ladrome-pages-json-v1-Markdown"

COLLECTION = chroma_client.get_or_create_collection(
    collection_name,
    metadata={"hnsw:space": "cosine"},
    embedding_function=emb_func)

CLIENT = OpenAI()

def get_context(query:str, n_results=10):
    result = COLLECTION.query(
        query_texts=query,
        n_results=n_results,
    )
    return result["documents"][0]

def get_response(query:str, context:List[str]):

    context = "\n\n###\n".join(context)
    messages = [
        {"role": "system", "content": "Tu es un assistant utile répondant aux questions. Tu dois répondre de manière précise et concise à la question de l'utilisateur. Si tu peux résumer sous forme de bullet points, fais-le. Si tu dispose d'un lien permettant d'avoir plus d'information sur le sujet, propose-le. Si la réponse n'est pas dans le contexte, alors dis simplement 'Je suis désolé, je n'ai pas la réponse à votre question"},
        {"role": "user", "content": "- Tu vouvoies l'utilisateur lorsque tu réponds à sa question.\n- Tu travailles au Conseil départemental de la Drôme.\n- Tu dois afficher les liens au format markdown et hypertexte.\n- Tu dois répondre à la question de l'utilisateur en une seule réponse. Tu ne peux pas lui reposer de questions derrière."},
        {"role": "user", "content": f"Contexte :\n\n {context}"},
        {"role": "user", "content": f"Question : {query}"},
    ]
    response = CLIENT.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [5]:
import pandas as pd 
test_queries = pd.read_excel("./data/Test Semantics V2.xlsx")
test_queries.head(50)

,Question,Réponse Wikit Semantics,Statut de la réponse,Commentaire,Commentaire Wikit
0,Comment obtenir la carte top départ ?,"Pour obtenir la carte Top Dép'Art, il y a deux...",ok,NaN,NaN
1,Comment obtenir l'APA ?,Pour obtenir l'Allocation Personnalisée Autono...,ko,manque les liens pr télécharger les docs + inf...,Indicateur 4/5 sur le fond de la réponse
2,qui dois je contacter pour faire une demande d...,Pour faire une demande d'APA (Allocation Perso...,ok,est-il possible de faire lien mailto ?,NaN
3,Comment devenir assistante maternelle ?,"Pour devenir assistante maternelle, vous devez...",ko,le contenu est en cours de creation,Indicateur 3/5 sur le fond de la réponse
4,Contacter la mdph,contacter la mdph\nAssistant conversationnel L...,ok,NaN,NaN
5,Où en est mon dossier AAH ?,Je ne sais pas. Veuillez contacter la Maison D...,ko,il faudrait que la réponse soit : Pour suivre ...,Indicateur 3/5 sur le fond de la réponse ; par...
6,question autour des offres d'emploi,"Je suis désolé, mais je ne peux pas répondre à...",ko,il faudrait répondre que pr postuler en ligne ...,Indicateur 1/5 : vérifier si on a le lien vers...
7,Où se situe le point d'accueil de la MDA à Mon...,Le point d'accueil de la MDA à Montélimar se s...,ko,il y a plusieurs adresses pr monté&limar donc ...,Indicateur 1/5
8,"J'ai perdu ma carte top départ, je fais commen...","Si vous avez perdu votre carte Top Dép'Art, vo...",ko,Aïe ! Tout n'est pas perdu !\nDéclarez rapidem...,Indicateur 5/5 : ce qui manque pour elle uniqu...
9,Il y a un problème sur le route départementale...,"Je suis désolé, mais en tant qu'Assistant conv...",ko,"mettre texte : ""Je suis désolé, mais en tant q...",Indicateur 1/05


In [10]:
query_id = 24
query = test_queries.iloc[query_id]["Question"]
#query = "Où puis-je trouver l'adresse des MDA qui ne sont pas à Valence ?"

print_query = True
print_context = False

if print_query:
    print(query)
context = get_context(query, 10)
if print_context:
    print(context)

response = get_response(query, context)
print(response)

je dois faire garder mes enfants
Voici les différents modes de garde disponibles pour vos enfants :

- Assistant(e) maternel(le) : Vous pouvez choisir de confier vos enfants à un(e) assistant(e) maternel(le) agréé(e) par le Département de la Drôme. Ces professionnels ont suivi une formation obligatoire de 120 heures et une initiation aux gestes de premiers secours. Ils sont répartis sur le territoire de la Drôme. [Guide pratique – Sécurité chez l’assistant(e) maternel(le)](https://www.ladrome.fr/wp-content/uploads/2020/01/guide-scuit-chez-ass-mat-pour-les-ass-mat.pdf)

- Établissements de placement : Si vous rencontrez des difficultés momentanées ou durables pour assumer la charge et l'éducation de vos enfants, vous pouvez envisager de les placer dans des établissements spécialisés tels que les villages d'enfants, les lieux de vie et d'accueil, les centres parentaux, les établissements sanitaires ou les établissements médico-sociaux d'éducation spéciale.

- Protection Maternelle et Inf

In [8]:
context

["Les bons plans sports nature\nLes sports nature font partie de l’ADN de la Drôme ! Randonnée pédestre, VTT, course d’orientation, escalade, trail, canoë, spéléo, parapente… presque tous les sports de nature peuvent se pratiquer dans la Drôme. Le Département œuvre au quotidien, en partenariat avec de nombreux acteurs, pour aménager des sites de pratique, les entretenir et concilier les usages. La Drôme compte aujourd’hui \n- environ 6 000 km de sentiers de randonnée (pédestre, équestre, VTT) \n- 500 km d’itinéraires d’activités aquatiques (canoë kayak) \n- 7 stations hiver/été \n- 5 via ferrata \n- une vingtaine d’Espaces Sports Orientation \n- une cinquantaine de sites de vol libre \n- plus d’une centaine de sites d’escalade \n- une centaine de cavités de spéléologie… Retrouvez tous les parcours et sentiers sur l’application mobile La Drôme et laissez-vous guider ! Accès à l’extranet sport nature\nPour plus d'informations:\n- Accès à l’extranet sport nature: https://collaboratif.ladr